In [1]:
from dt_apriltags import Detector
import cv2
import numpy as np
import open3d as o3d
import json
import glob
import matplotlib.pyplot as plt
import copy
import teaserpp_python
from numpy.linalg import inv
from scipy.spatial import cKDTree
import os
import time
from numba import jit, prange

def detect_tag(img,k,tag_size):
    at_detector = Detector(families='tagStandard41h12',
                       nthreads=8,
                       quad_decimate=1.0,
                       quad_sigma=0.8,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)
    tags  = at_detector.detect(img,True,k,tag_size)
    #print("detected : ", len(tags), " tags")
    best_tag = min(tags, key=lambda tag: tag.pose_err)
    # create a 4x4 identity matrix
    H = np.eye(4)

    # set the rotation
    H[:3, :3] = best_tag.pose_R

    # set the translation
    H[:3, 3] = best_tag.pose_t[:, 0]
    #print("Done detecting tags")

    return H, best_tag.tag_id

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def register_two_views_teaser(A_pcd_raw,B_pcd_raw,VOXEL_SIZE):
    
    VISUALIZE = True
    A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    #if VISUALIZE:
     #   o3d.visualization.draw_geometries([A_pcd,B_pcd]) # plot downsampled A and B 

    A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
    B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

    print("Extracting FPFH features")
    # extract FPFH features
    A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
    B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)
    print(A_feats.shape)
    print("Computing FPFH correspondences")
    # establish correspondences by nearest neighbour search in feature space
    corrs_A, corrs_B = find_correspondences(
        A_feats, B_feats, mutual_filter=True)
    A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
    B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

    num_corrs = A_corr.shape[1]
    print(f'FPFH generates {num_corrs} putative correspondences.')

    # visualize the point clouds together with feature correspondenc
    # robust global registration using TEASER++
    NOISE_BOUND = VOXEL_SIZE
    teaser_solver = get_teaser_solver(NOISE_BOUND)
    teaser_solver.solve(A_corr,B_corr)
    solution = teaser_solver.getSolution()
    R_teaser = solution.rotation
    t_teaser = solution.translation
    T_teaser = Rt2T(R_teaser,t_teaser)

    # Visualize the registration results
    A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
    #o3d.visualization.draw_geometries([A_pcd_T_teaser,B_pcd])

    # local refinement using ICP
    icp_sol = o3d.pipelines.registration.registration_icp(
          A_pcd, B_pcd, NOISE_BOUND, T_teaser,
          o3d.pipelines.registration.TransformationEstimationPointToPoint(),
          o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
    T_icp = icp_sol.transformation

    # visualize the registration after ICP refinement
    A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
    if VISUALIZE:
        Acopy = copy.deepcopy(A_pcd_T_icp).paint_uniform_color([0.0,0.0,1])
        Bcopy = copy.deepcopy(B_pcd).paint_uniform_color([1.0,0.0,0.0])
        o3d.visualization.draw_geometries([Acopy,Bcopy])
    tformed_A = copy.deepcopy(A_pcd_raw).transform(T_icp)
    res = o3d.geometry.PointCloud()
    res = tformed_A + B_pcd_raw
    
    return res,T_icp

def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def extract_fpfh(pcd, voxel_size):
    radius_normal = voxel_size * 2
    pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return np.array(fpfh.data).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
    feat1tree = cKDTree(feat1)
    dists, nn_inds = feat1tree.query(feat0, k=knn, workers=10)
    if return_distance:
        return nn_inds, dists
    else:
        return nn_inds

def find_correspondences(feats0, feats1, mutual_filter=True):
    nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
    corres01_idx0 = np.arange(len(nns01))
    corres01_idx1 = nns01

    if not mutual_filter:
        return corres01_idx0, corres01_idx1

    nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
    corres10_idx1 = np.arange(len(nns10))
    corres10_idx0 = nns10

    mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
    corres_idx0 = corres01_idx0[mutual_filter]
    corres_idx1 = corres01_idx1[mutual_filter]

    return corres_idx0, corres_idx1

def get_teaser_solver(noise_bound):
    solver_params = teaserpp_python.RobustRegistrationSolver.Params()
    solver_params.cbar2 = 1.0
    solver_params.noise_bound = noise_bound
    solver_params.estimate_scaling = False
    solver_params.inlier_selection_mode = \
        teaserpp_python.RobustRegistrationSolver.INLIER_SELECTION_MODE.PMC_EXACT
    solver_params.rotation_tim_graph = \
        teaserpp_python.RobustRegistrationSolver.INLIER_GRAPH_FORMULATION.CHAIN
    solver_params.rotation_estimation_algorithm = \
        teaserpp_python.RobustRegistrationSolver.ROTATION_ESTIMATION_ALGORITHM.GNC_TLS
    solver_params.rotation_gnc_factor = 1.4
    solver_params.rotation_max_iterations = 10000
    solver_params.rotation_cost_threshold = 1e-16
    solver = teaserpp_python.RobustRegistrationSolver(solver_params)
    return solver

def Rt2T(R,t):
    T = np.identity(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T 

def eliminate_flying_pixels(depth_image, ws, threshold):
   
    # Get image size
    height, width = depth_image.shape
    # Create an empty array for the result
    result = np.zeros_like(depth_image, dtype=float)

    # Iterate over the entire image
    for cy in range(height):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            # Get the window
            window = depth_image[y_start:y_end, x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    count = np.sum(result > threshold)

    depth_image[result > threshold] = 0
    return  depth_image



def colored_ICP(source, target):
    
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        print("iteration: ", iters, radius, scale)

        print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = copy.deepcopy(source).voxel_down_sample(radius)
        target_down = copy.deepcopy(target).voxel_down_sample(radius)

        print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = result_icp.transformation
    
   
        draw_registration_result(source, target, current_transformation)
    return current_transformation

def backproject_o3d(rgbd_frame, intrinsics):
    
    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgbd_frame.color, rgbd_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)


    # Create a point cloud from the RGBD image
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc, intrinsics)
    n_radius = 0.01*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=n_radius, max_nn=30))
    # Visualize the point cloud
    #o3d.visualization.draw_geometries([pcd])
    
    return pcd

@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    #for i in prange(height):
       # for j in prange(width):
      #      if result[i, j] > threshold:
     #           depth_image[i, j] = 0
    #count = np.sum(result > threshold)
    #print("Numba detected: #", count)
    return result


def load_filter_pcds(data_path,flying_pixel_filter_threshold):  # returns a list of  pcds


    l = 0.3048  # replace with the actual value
    ws = 2
   
    # define a dictionary where the keys are the tag_ids and the values are the transformation matrices
    transformations = {
        0: np.array([[0, 1, 0, 0],
                    [1, 0, 0, 0],
                    [0, 0, -1, l/2],
                    [0, 0, 0, 1]]),
        1: np.array([[0, 0, -1, l/2],
                    [1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]]),
        2: np.array([[-1, 0, 0, 0],
                    [0, 0, -1, l/2],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]]),
        3: np.array([[0, 0, 1, -l/2],
                    [-1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]]),
        4: np.array([[1, 0, 0, 0],
                    [0, 0, 1, -l/2],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]])
    }
    
    tag_size = 0.145    # assuming best_tag is defined and contains the tag with the smallest pose_err

    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    
        
    files = glob.glob(data_path+'/*.mkv')
    files.sort()
    #print("There are : ", len(files), "present here!")

    list_size = len(files)
    rgbd_frames = [None] * list_size

    tag_poses = [None] * list_size  # assuming tag_poses and current_transform are dictionaries
    current_transform = [None] * list_size
    pcds = []
    for i in range(len(files)): # for each view
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        #print("Current File: ", file_name)

        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)
        K = (fx,fy,cx,cy)

        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame()
            if rgbda is None:
                #print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            #print("Got the last frame")
            rgbd_frames[i] = last_frame
        else:
            
            print("************No valid frames found in the .mkv file.**********")

        rgb_im_np = np.asarray(last_frame.color)
    
        gray_img = cv2.cvtColor(rgb_im_np, cv2.COLOR_RGB2GRAY)
        im_name = "img_" + str(i) + ".jpg" 
        cv2.imwrite(im_name, gray_img)
        transform, tag_id = detect_tag(gray_img,K, tag_size)
        #print("T: ", transform)
        #print("ID: ", tag_id)
        
        # get the transformation matrix for the given tag_id
        cube_transform = transformations.get(tag_id, np.eye(4))  # defaults to identity matrix if tag_id is not found

        tag_poses[i] = np.dot(transform, np.linalg.inv(cube_transform))
        
        # assuming tag_poses[0] is defined and is the pose of the first camera
        tag_pose = np.dot(tag_poses[0], np.linalg.inv(tag_poses[i]))

        current_transform[i] = tag_pose  # current_transform is now the tag pose
        reader.close()
        fname_tform = "H_0_" + str(i) + ".txt"
        np.savetxt(data_path+fname_tform, current_transform[i])
        depth_image_array = np.asarray(last_frame.depth) # reference the same depth so it will change
         
        start_time_numba = time.time()
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        end_time_numba = time.time()

        # apply thresholding
        #count = np.sum(result_mask > flying_pixel_filter_threshold)
        #print("Numba Detected :#", count)

        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        filtered_depth = depth_image_array
        execution_time_numba = end_time_numba - start_time_numba
        #filtered_depth = bilateral_filter(depth_image_array)
        last_frame.depth = o3d.geometry.Image(filtered_depth)
        pcd = backproject_o3d(last_frame, camera_intrinsics)
        #o3d.visualization.draw_geometries([pcd])
        #break
        pcds.append(copy.deepcopy(pcd).transform(current_transform[i]))

    #o3d.visualization.draw_geometries(pcds)
    return pcds,current_transform


def compute_and_orient_normals(pcd, voxel_size):
    normal_radius = voxel_size*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(normal_radius=0.1, max_nn=30))
    
data_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_calib_new4/'
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
start_time = time.time()
#pcds, tforms = load_filter_pcds(data_path,1200) # lower threshold will remove more points()
end_time = time.time()
execution_time = end_time - start_time
print("It took :", execution_time, "s to process all frames")

print("Created " ,len(pcds), " point clouds")

# t01= colored_ICP(pcds[0],pcds[1])
# t52= colored_ICP(pcds[5],pcds[2])
# t43= colored_ICP(pcds[4],pcds[3])
# t12= colored_ICP(pcds[1],pcds[2])
# t32= colored_ICP(pcds[3],pcds[2])


# H0 = t12 @ t01
# H1 = t12
# H3 = t32
# H4 = t32 @ t43
# H5 = t52


# np.savetxt("c_icp_0_2.txt", H0)
# np.savetxt("c_icp_1_2.txt", H1)
# np.savetxt("c_icp_3_2.txt", H3)
# np.savetxt("c_icp_4_2.txt", H4)
# np.savetxt("c_icp_5_2.txt", H5)

# p0 = copy.deepcopy(pcds[0]).transform(H0)
# p1 = copy.deepcopy(pcds[1]).transform(H1)
# p3 = copy.deepcopy(pcds[3]).transform(H3)
# p4 = copy.deepcopy(pcds[4]).transform(H4)
# p5 = copy.deepcopy(pcds[5]).transform(H5)

# o3d.visualization.draw_geometries([p0,p1,pcds[2],p3,p4,p5])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
It took : 9.775161743164062e-06 s to process all frames


NameError: name 'pcds' is not defined

In [ ]:
def load_transformations(tform_path):
    """Load transformations from the given path."""
    htm_files = ["htm_0_3.txt", "htm_1_3.txt", "htm_5_3.txt", "htm_6_3.txt", "htm_7_3.txt"]
    return [np.loadtxt(tform_path + file) for file in htm_files]

In [ ]:
tform_path = 

In [ ]:
from dt_apriltags import Detector
import cv2
import numpy as np
import open3d as o3d
import json
import glob
import matplotlib.pyplot as plt
import copy
import teaserpp_python
from numpy.linalg import inv
from scipy.spatial import cKDTree
import os
import time
from numba import jit, prange
import argparse

def detect_tag(img,k,tag_size):
    at_detector = Detector(families='tagStandard41h12',
                       nthreads=8,
                       quad_decimate=1.0,
                       quad_sigma=0.8,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)
    tags  = at_detector.detect(img,True,k,tag_size)
    #print("detected : ", len(tags), " tags")
    best_tag = min(tags, key=lambda tag: tag.pose_err)
    # create a 4x4 identity matrix
    H = np.eye(4)

    # set the rotation
    H[:3, :3] = best_tag.pose_R

    # set the translation
    H[:3, 3] = best_tag.pose_t[:, 0]
    #print("Done detecting tags")

    return H, best_tag.tag_id

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def register_two_views_teaser(A_pcd_raw,B_pcd_raw,VOXEL_SIZE):
    
    VISUALIZE = True
    A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    #if VISUALIZE:
     #   o3d.visualization.draw_geometries([A_pcd,B_pcd]) # plot downsampled A and B 

    A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
    B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

    print("Extracting FPFH features")
    # extract FPFH features
    A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
    B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)
    print(A_feats.shape)
    print("Computing FPFH correspondences")
    # establish correspondences by nearest neighbour search in feature space
    corrs_A, corrs_B = find_correspondences(
        A_feats, B_feats, mutual_filter=True)
    A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
    B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

    num_corrs = A_corr.shape[1]
    print(f'FPFH generates {num_corrs} putative correspondences.')

    # visualize the point clouds together with feature correspondenc
    # robust global registration using TEASER++
    NOISE_BOUND = VOXEL_SIZE
    teaser_solver = get_teaser_solver(NOISE_BOUND)
    teaser_solver.solve(A_corr,B_corr)
    solution = teaser_solver.getSolution()
    R_teaser = solution.rotation
    t_teaser = solution.translation
    T_teaser = Rt2T(R_teaser,t_teaser)

    # Visualize the registration results
    A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
    #o3d.visualization.draw_geometries([A_pcd_T_teaser,B_pcd])

    # local refinement using ICP
    icp_sol = o3d.pipelines.registration.registration_icp(
          A_pcd, B_pcd, NOISE_BOUND, T_teaser,
          o3d.pipelines.registration.TransformationEstimationPointToPoint(),
          o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
    T_icp = icp_sol.transformation

    # visualize the registration after ICP refinement
    A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
    if VISUALIZE:
        Acopy = copy.deepcopy(A_pcd_T_icp).paint_uniform_color([0.0,0.0,1])
        Bcopy = copy.deepcopy(B_pcd).paint_uniform_color([1.0,0.0,0.0])
        o3d.visualization.draw_geometries([Acopy,Bcopy])
    tformed_A = copy.deepcopy(A_pcd_raw).transform(T_icp)
    res = o3d.geometry.PointCloud()
    res = tformed_A + B_pcd_raw
    
    return res,T_icp

def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def extract_fpfh(pcd, voxel_size):
    radius_normal = voxel_size * 2
    pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return np.array(fpfh.data).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
    feat1tree = cKDTree(feat1)
    dists, nn_inds = feat1tree.query(feat0, k=knn, workers=10)
    if return_distance:
        return nn_inds, dists
    else:
        return nn_inds

def find_correspondences(feats0, feats1, mutual_filter=True):
    nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
    corres01_idx0 = np.arange(len(nns01))
    corres01_idx1 = nns01

    if not mutual_filter:
        return corres01_idx0, corres01_idx1

    nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
    corres10_idx1 = np.arange(len(nns10))
    corres10_idx0 = nns10

    mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
    corres_idx0 = corres01_idx0[mutual_filter]
    corres_idx1 = corres01_idx1[mutual_filter]

    return corres_idx0, corres_idx1

def get_teaser_solver(noise_bound):
    solver_params = teaserpp_python.RobustRegistrationSolver.Params()
    solver_params.cbar2 = 1.0
    solver_params.noise_bound = noise_bound
    solver_params.estimate_scaling = False
    solver_params.inlier_selection_mode = \
        teaserpp_python.RobustRegistrationSolver.INLIER_SELECTION_MODE.PMC_EXACT
    solver_params.rotation_tim_graph = \
        teaserpp_python.RobustRegistrationSolver.INLIER_GRAPH_FORMULATION.CHAIN
    solver_params.rotation_estimation_algorithm = \
        teaserpp_python.RobustRegistrationSolver.ROTATION_ESTIMATION_ALGORITHM.GNC_TLS
    solver_params.rotation_gnc_factor = 1.4
    solver_params.rotation_max_iterations = 10000
    solver_params.rotation_cost_threshold = 1e-16
    solver = teaserpp_python.RobustRegistrationSolver(solver_params)
    return solver

def Rt2T(R,t):
    T = np.identity(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T 

def eliminate_flying_pixels(depth_image, ws, threshold):
   
    # Get image size
    height, width = depth_image.shape
    # Create an empty array for the result
    result = np.zeros_like(depth_image, dtype=float)

    # Iterate over the entire image
    for cy in range(height):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            # Get the window
            window = depth_image[y_start:y_end, x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    count = np.sum(result > threshold)

    depth_image[result > threshold] = 0
    return  depth_image



def colored_ICP(source, target):
    
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        print("iteration: ", iters, radius, scale)

        print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = copy.deepcopy(source).voxel_down_sample(radius)
        target_down = copy.deepcopy(target).voxel_down_sample(radius)

        print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = result_icp.transformation
    
   
        draw_registration_result(source, target, current_transformation)
    return current_transformation

def backproject_o3d(rgbd_frame, intrinsics):
    
    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgbd_frame.color, rgbd_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)


    # Create a point cloud from the RGBD image
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc, intrinsics)
    n_radius = 0.01*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=n_radius, max_nn=30))
    # Visualize the point cloud
    #o3d.visualization.draw_geometries([pcd])
    
    return pcd

@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    #for i in prange(height):
       # for j in prange(width):
      #      if result[i, j] > threshold:
     #           depth_image[i, j] = 0
    #count = np.sum(result > threshold)
    #print("Numba detected: #", count)
    return result


def load_filter_pcds(data_path,flying_pixel_filter_threshold):  # returns a list of  pcds


    l = 0.3048  # replace with the actual value
    ws = 2
   
    # define a dictionary where the keys are the tag_ids and the values are the transformation matrices
    transformations = {
        0: np.array([[0, 1, 0, 0],
                    [1, 0, 0, 0],
                    [0, 0, -1, l/2],
                    [0, 0, 0, 1]]),
        1: np.array([[0, 0, -1, l/2],
                    [1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]]),
        2: np.array([[-1, 0, 0, 0],
                    [0, 0, -1, l/2],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]]),
        3: np.array([[0, 0, 1, -l/2],
                    [-1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]]),
        4: np.array([[1, 0, 0, 0],
                    [0, 0, 1, -l/2],
                    [0, -1, 0, 0],
                    [0, 0, 0, 1]])
    }
    
    tag_size = 0.145    # assuming best_tag is defined and contains the tag with the smallest pose_err

    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    
        
    files = glob.glob(data_path+'/*.mkv')
    files.sort()
    #print("There are : ", len(files), "present here!")

    list_size = len(files)
    rgbd_frames = [None] * list_size

    tag_poses = [None] * list_size  # assuming tag_poses and current_transform are dictionaries
    current_transform = [None] * list_size
    pcds = []
    for i in range(len(files)): # for each view
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        #print("Current File: ", file_name)

        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)
        K = (fx,fy,cx,cy)

        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame()
            if rgbda is None:
                #print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            #print("Got the last frame")
            rgbd_frames[i] = last_frame
        else:
            
            print("************No valid frames found in the .mkv file.**********")

        rgb_im_np = np.asarray(last_frame.color)
    
        gray_img = cv2.cvtColor(rgb_im_np, cv2.COLOR_RGB2GRAY)
        im_name = "img_" + str(i) + ".jpg" 
        cv2.imwrite(im_name, gray_img)
        transform, tag_id = detect_tag(gray_img,K, tag_size)
        #print("T: ", transform)
        #print("ID: ", tag_id)
        
        # get the transformation matrix for the given tag_id
        cube_transform = transformations.get(tag_id, np.eye(4))  # defaults to identity matrix if tag_id is not found

        tag_poses[i] = np.dot(transform, np.linalg.inv(cube_transform))
        
        # assuming tag_poses[0] is defined and is the pose of the first camera
        tag_pose = np.dot(tag_poses[0], np.linalg.inv(tag_poses[i]))

        current_transform[i] = tag_pose  # current_transform is now the tag pose
        reader.close()
        # WRITE THE TRANSFORMATIONS
        fname_tform = "H_0_" + str(i) + ".txt"
        np.savetxt(data_path+fname_tform, current_transform[i])
        depth_image_array = np.asarray(last_frame.depth) # reference the same depth so it will change
         
        #start_time_numba = time.time()
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        #end_time_numba = time.time()

        # apply thresholding
        #count = np.sum(result_mask > flying_pixel_filter_threshold)
        #print("Numba Detected :#", count)

        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        filtered_depth = depth_image_array
        #execution_time_numba = end_time_numba - start_time_numba
        #filtered_depth = bilateral_filter(depth_image_array)
        last_frame.depth = o3d.geometry.Image(filtered_depth)
        pcd = backproject_o3d(last_frame, camera_intrinsics)
        #o3d.visualization.draw_geometries([pcd])
        #break
        pcds.append(copy.deepcopy(pcd).transform(current_transform[i]))

    o3d.visualization.draw_geometries(pcds)
    return pcds,current_transform


def compute_and_orient_normals(pcd, voxel_size):
    normal_radius = voxel_size*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(normal_radius=0.1, max_nn=30))
    


if __name__ == "__main__":
    #parser = argparse.ArgumentParser(description="Perform Azure Kinect calibration using the apriltag cube.")
    #parser.add_argument("-i", "--input", required=True, help="Path to the calibration data.")
    #args = parser.parse_args()
    
    #main(args.input, args.transform)
    data_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_calib_new4/'
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
    start_time = time.time()
    pcds, tforms = load_filter_pcds(data_path,1200) # lower threshold will remove more points()
    end_time = time.time()
    execution_time = end_time - start_time
    print("It took :", execution_time, "s to process all frames")

    print("Created " ,len(pcds), " point clouds")
    #o3d.visualization.draw_geometries(pcds)

# t01= colored_ICP(pcds[0],pcds[1])
# t52= colored_ICP(pcds[5],pcds[2])
# t43= colored_ICP(pcds[4],pcds[3])
# t12= colored_ICP(pcds[1],pcds[2])
# t32= colored_ICP(pcds[3],pcds[2])


# H0 = t12 @ t01
# H1 = t12
# H3 = t32
# H4 = t32 @ t43
# H5 = t52


# np.savetxt("c_icp_0_2.txt", H0)
# np.savetxt("c_icp_1_2.txt", H1)
# np.savetxt("c_icp_3_2.txt", H3)
# np.savetxt("c_icp_4_2.txt", H4)
# np.savetxt("c_icp_5_2.txt", H5)

# p0 = copy.deepcopy(pcds[0]).transform(H0)
# p1 = copy.deepcopy(pcds[1]).transform(H1)
# p3 = copy.deepcopy(pcds[3]).transform(H3)
# p4 = copy.deepcopy(pcds[4]).transform(H4)
# p5 = copy.deepcopy(pcds[5]).transform(H5)

# o3d.visualization.draw_geometries([p0,p1,pcds[2],p3,p4,p5])


In [ ]:
import numpy as np
import threading

def eliminate_flying_pixels_worker(depth_image, ws, threshold, y_start, y_end, result):
    """Worker function to process a chunk of the image."""
    
    height, width = depth_image.shape
    
    # Iterate over the chunk of the image assigned to this thread
    for cy in range(y_start, y_end):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            
            # Get the window
            window = depth_image[max(0, cy - ws):min(height, cy + ws + 1), x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))

def parallel_eliminate_flying_pixels(depth_image, ws, threshold, num_threads=4):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=float)
    
    # Create threads and split the image into chunks for each thread
    threads = []
    chunk_size = height // num_threads
    for i in range(num_threads):
        y_start = i * chunk_size
        y_end = (i + 1) * chunk_size if i != num_threads - 1 else height  # Handle the last chunk which might be bigger
        thread = threading.Thread(target=eliminate_flying_pixels_worker, 
                                  args=(depth_image, ws, threshold, y_start, y_end, result))
        threads.append(thread)
        thread.start()
    
    # Wait for all threads to finish
    for thread in threads:
        thread.join()
    
    # Apply the threshold to the depth image
    depth_image[result > threshold] = 0
    count = np.sum(result > threshold)
    print("Parallel detected: #", count)
    return depth_image

# Dummy data for testing
#depth_image = np.random.rand(100, 100)
#ws = 2
#threshold = 1200

# Test the function
#output_image = parallel_eliminate_flying_pixels(depth_image, ws, threshold, os.cpu_count())


from numba import jit, prange

@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    #for i in prange(height):
       # for j in prange(width):
      #      if result[i, j] > threshold:
     #           depth_image[i, j] = 0
    #count = np.sum(result > threshold)
    #print("Numba detected: #", count)
    return result

# Let's time the Numba version

#execution_time_numba



In [ ]:
from scipy.signal import convolve2d
def fast_s1_with_threshold(depth, WS, threshold):
    """
    Calculate the s1 scores for the entire depth image using convolution and identify flying pixels.
    
    Parameters:
    - depth: 2D numpy array representing the depth image.
    - WS: Integer, the window size.
    - threshold: Float, the threshold value for identifying flying pixels.
    
    Returns:
    - s1_scores: 2D numpy array of the same shape as depth, containing the s1 scores.
    - flying_pixels: 2D numpy array of the same shape as depth, where 1 indicates a flying pixel and 0 otherwise.
    """
    
    # Create a kernel of ones with size (2*WS + 1, 2*WS + 1)
    kernel = np.ones((2*WS + 1, 2*WS + 1))
    kernel[WS, WS] = 0  # set the center pixel to 0
    
    # Compute the difference between each pixel and its neighbors
    diff = convolve2d(depth, kernel, mode='same', boundary='symm')
    
    # Compute the s1 scores by summing the absolute differences within the window
    s1_scores = convolve2d(np.abs(diff), np.ones((2*WS + 1, 2*WS + 1)), mode='same', boundary='symm')
    
    # Identify flying pixels based on the threshold
    flying_pixels = (s1_scores > threshold).astype(int)
    
    return s1_scores, flying_pixels

# Test the function with a threshold value of 10
#test_s1_scores, test_flying_pixels = fast_s1_with_threshold(test_depth_large, 3, 10)

# Display the number of identified flying pixels
#np.sum(test_flying_pixels)


In [ ]:
# THIS REPAIRS FLYING PIXELS
def eliminate_flying_pixels_replace_v1(depth_image, ws, threshold):
    # Get image size
    height, width = depth_image.shape
    # Create an empty array for the result
    result = np.zeros_like(depth_image, dtype=float)
    count = 0
    cor = 0
    nope=0
    # Function to get Euclidean distance between two points
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    # Iterate over the entire image
    for cy in range(height):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            # Get the window
            window = depth_image[y_start:y_end, x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
            if result[cy, cx] > threshold:
                count += 1
                window_result = result[y_start:y_end, x_start:x_end]

                valid_pixels = np.where(window_result <= threshold)
                valid_coords = list(zip(*valid_pixels))

                if valid_coords:
                    distances = [(cy-y)**2 + (cx-x)**2 for y, x in valid_coords]
                    nearest_y, nearest_x = valid_coords[np.argmin(distances)]
                    depth_image[cy, cx] = 0#depth_image[y_start + nearest_y, x_start + nearest_x]
                    cor+=1
                else:
                    depth_image[cy, cx] = 0
                    nope+=1
    print("Detected: #",count, " flying pixels")   
    print("Corrected: #",cor, " flying pixels")
    print("Invalidated: #",nope, " flying pixels")       
    return depth_image

In [ ]:
data_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_calib_new4//'
pcds, tforms = load_filter_pcds(data_path,100000) # lower threshold will remove more points()


In [ ]:
def eliminate_flying_pixels_old(depth_image, ws, threshold):
   
    # Get image size
    height, width = depth_image.shape
    # Create an empty array for the result
    result = np.zeros_like(depth_image, dtype=float)

    # Iterate over the entire image
    for cy in range(height):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            # Get the window
            window = depth_image[y_start:y_end, x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    count = np.sum(result > threshold)
    print("Detected : #", count, " flying pixels -old")
    depth_image[result > threshold] = 0
    return  depth_image

In [ ]:
# FAST? CORRECT - 224229
from scipy.spatial import cKDTree
import numpy as np
from scipy import ndimage


def eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=float)

    # Vectorized window-based computation
    for cy in range(height):
        for cx in range(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))

    # Create KD-Tree for faster nearest neighbor search
    y_indices, x_indices = np.where(result <= threshold)
    valid_points = np.c_[y_indices, x_indices]
    tree = cKDTree(valid_points)

    # Get the coordinates of the pixels we want to replace
    y_bad, x_bad = np.where(result > threshold)
    distances, indices = tree.query(np.c_[y_bad, x_bad], k=2)
    count = np.sum(result > threshold)
    print("Detected: #",count, " flying pixels")         


    # Replace bad pixels with the values of the nearest valid pixels
    depth_image[y_bad, x_bad] = depth_image[y_indices[indices], x_indices[indices]]
    return depth_image

# IMPROVED?


def eliminate_flying_pixels_fast_v1(depth_image, ws, threshold):
    # Generate a window kernel
    kernel = np.ones((2*ws+1, 2*ws+1))

    # Compute the average in the window for each pixel
    avg_in_window = ndimage.convolve(depth_image, kernel, mode='constant') / (2*ws+1)**2

    # Calculate the absolute differences between each pixel and the window average
    abs_diff = np.abs(depth_image - avg_in_window)

    # Create a mask for values over the threshold
    mask = abs_diff > threshold

    # Set flying pixels to zero
    depth_image[mask] = 0

    return depth_image

def eliminate_flying_pixels_fast_v2(depth_image, ws, threshold):
    # Generate a window kernel
    kernel = np.ones((2*ws+1, 2*ws+1))

    # Calculate the sum of absolute differences using convolution
    sad = ndimage.convolve(np.abs(depth_image - ndimage.convolve(depth_image, kernel, mode='constant')), kernel, mode='constant')

    # Create a mask for values over the threshold
    mask = sad > threshold

    # Set flying pixels to zero
    depth_image[mask] = 0

    return depth_image



In [ ]:
import numpy as np

def eliminate_flying_pixels_single_loop(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=float)

    # Compute the rectangular window limits for every pixel in one go
    y_start = np.clip(np.arange(height).reshape(-1, 1) - ws, 0, height)
    y_end = np.clip(np.arange(height).reshape(-1, 1) + ws + 1, 0, height)
    x_start = np.clip(np.arange(width) - ws, 0, width)
    x_end = np.clip(np.arange(width) + ws + 1, 0, width)


    # Count of flying pixels
    count = 0

    for cy in range(height):
        for cx in range(width):
            window = depth_image[y_start[cy, cx]:y_end[cy, cx], x_start[cy, cx]:x_end[cy, cx]]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))

            iif result[cy, cx] > threshold:
    count += 1
    window_y_start, window_y_end = y_start[cy], y_end[cy]
    window_x_start, window_x_end = x_start[cx], x_end[cx]
    window_result = result[window_y_start:window_y_end, window_x_start:window_x_end]
    
    valid_pixels = np.where(window_result <= threshold)
    valid_coords = list(zip(*valid_pixels))

    if valid_coords:
        distances = [(cy-y)**2 + (cx-x)**2 for y, x in valid_coords]
        nearest_y, nearest_x = valid_coords[np.argmin(distances)]
        depth_image[cy, cx] = depth_image[window_y_start + nearest_y, window_x_start + nearest_x]
    else:
        depth_image[cy, cx] = 0


    print("Detected: #", count, "flying pixels")
    return depth_image


In [ ]:
# THIS REPAIRS FLYING PIXELS
def eliminate_flying_pixels(depth_image, ws, threshold):
    # Get image size
    height, width = depth_image.shape
    # Create an empty array for the result
    result = np.zeros_like(depth_image, dtype=float)

    # Function to get Euclidean distance between two points
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    # Iterate over the entire image
    for cy in range(height):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            # Get the window
            window = depth_image[y_start:y_end, x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    count = np.sum(result > threshold)
    # Loop over the result matrix to replace values beyond the threshold
    for cy in range(height):
        for cx in range(width):
            if result[cy, cx] > threshold:
                min_distance = float('inf')
                nearest_value = None
                
                x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
                y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)

                # Loop over the window to find the nearest valid depth value
                for y in range(y_start, y_end):
                    for x in range(x_start, x_end):
                        if result[y, x] <= threshold and distance((cy, cx), (y, x)) < min_distance:
                            min_distance = distance((cy, cx), (y, x))
                            nearest_value = depth_image[y, x]
                
                # If no valid pixel is found in the neighborhood, set the pixel to 0
                depth_image[cy, cx] = nearest_value if nearest_value is not None else 0
    print("Detected: #",count, " flying pixels")         
    return depth_image


In [ ]:
A_corr.shape

In [ ]:
correspondences = {}
for i, pc_i in enumerate(point_clouds):
    for j, pc_j in enumerate(point_clouds):
        if i != j:
            print("Establishing correspondences between: (", i ," and ", j,")",)
            A_corr, B_corr = compute_pairwise_corrs(pc_i, pc_j, VOXEL_SIZE=0.05)
            correspondences[(i, j)] = (A_corr, B_corr)


In [ ]:
#Hi = np.eye(4)
#Hi[:3, :3] 
Hi[:3, 3] = np.array([3,4,5])

In [ ]:
Hi

In [ ]:

num_point_clouds = len(point_clouds)
homogeneous_transformations = [np.eye(4) for _ in range(num_point_clouds)]


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.optimize import least_squares

def robust_loss(e,mu):
    return  mu * e**2 / (mu**2 + e**2)

def objective_function(H, X, correspondences,mu):
    homogeneous_transformations = unpack_params(H)
    J = 0
   
    for (i, j), (A_corr, B_corr) in correspondences.items():
        w_H_i = homogeneous_transformations[i]
        w_H_j = homogeneous_transformations[j]
        C_i = np.vstack((A_corr, np.ones(A_corr.shape[1]))) # Homogeneous coordinates
        C_j = np.vstack((B_corr, np.ones(B_corr.shape[1]))) # Homogeneous coordinates

        transformed_C_i = w_H_i.dot(C_i)
        transformed_C_j_inv = np.linalg.inv(w_H_j).dot(C_j)

        error = transformed_C_j_inv - transformed_C_i
        error = np.sum(error[:3] ** 2, axis=0) # Ignoring homogeneous component, summing squared differences

        J += np.sum(robust_loss(error,mu))
        #print(J)

    return J


def error_euc_constrained(params, X, corrs, mu):
    H = unpack_params(params)
    res = []
    for i in range(len(X)):
        for j in range(len(X)):
            if i == j:
                continue
            
            iHj = np.linalg.inv(H[i]) @ H[j]
            Xi_est = iHj[:3, :3] @ X[j] + iHj[:3, 3, None]

            xi_est = Xi_est[:, corrs[i][j][:, 1]]
            xi = X[i][:, corrs[i][j][:, 0]]
            e = xi - xi_est
            res_i = mu * e**2 / (mu**2 + e**2)
            res.extend(res_i.ravel())
    return np.array(res)

def unpack_params(params): # From EULER to HTM
    H = [np.eye(4)]
    for i in range(0, len(params), 6):
        rpy = params[i:i+3]
        t = params[i+3:i+6]
        rotation_matrix = R.from_euler('zyx', rpy, degrees=False).as_matrix()
        Hi = np.eye(4)
        Hi[:3, :3] = rotation_matrix
        Hi[:3, 3] = t
        H.append(Hi)
    return H

def pack_params(H): # TO EULER
    params = []
    for i in range(1, len(H)):
        rotation_matrix = H[i][:3, :3]
        rpy = R.from_matrix(rotation_matrix).as_euler('zyx', degrees=False)
        t = H[i][:3, 3]
        params.extend(list(rpy) + list(t))
    return params

def bundle_registration_6dof(H,X, corrs):
    options = {'ftol': 1e-5, 'xtol': 1e-3, 'verbose': 1}
    mu_values = [1, 0.7143, 0.5102, 0.3644, 0.2603, 0.1859, 0.1328, 0.0949, 0.0678, 0.0484, 0.0346, 0.0247]

    for mu in mu_values:
        input_params = pack_params(H)
        result = least_squares(objective_function, input_params, args=(X, corrs,mu), **options)
        H = unpack_params(result.x)
    return H


In [ ]:
homogeneous_transformations

In [ ]:
htms_init[0]

In [ ]:
htms_init = [H0,H1,np.eye(4),H3,H4,H5]

In [ ]:
dd  = pack_params(htms_init)

In [ ]:
H_set = bundle_registration_6dof(htms_init,X, correspondences)

In [ ]:
H_set

In [ ]:
q0 = copy.deepcopy(p0).transform(H_set[0])
q1 = copy.deepcopy(p1).transform(H_set[1])
q3 = copy.deepcopy(p3).transform(H_set[3])
q4 = copy.deepcopy(p4).transform(H_set[4])
q5 = copy.deepcopy(p5).transform(H_set[5])

In [ ]:
o3d.visualization.draw_geometries([q0,q1,q3,q4,q5,boxes[2]])

In [ ]:
p = pack_params(homogeneous_transformations)

In [ ]:
X = [np.asarray(pcd.points).T for pcd in point_clouds]

In [ ]:
H = bundle_registration_6dof(homogeneous_transformations,X,correspondences)

In [ ]:
def compute_pairwise_corrs(p0,p1,VOXEL_SIZE=0.05):    
    VOXEL_SIZE = 0.05
    pp0 = p0.voxel_down_sample(voxel_size=VOXEL_SIZE)
    pp1 = p1.voxel_down_sample(voxel_size=VOXEL_SIZE)

    A_xyz = pcd2xyz(pp0) # np array of size 3 by N
    B_xyz = pcd2xyz(pp1) # np array of size 3 by M


    # The XYZ points are supposed to be the matches, lets transpose and find the mutual nearest euclidean neighbors
    A_feats = A_xyz.T
    B_feats = B_xyz.T

    #print(A_feats.shape)
    #print("Computing Nearest Neighbor correspondences")
    # establish correspondences by nearest neighbour search in feature space
    corrs_A, corrs_B = find_correspondences(
        A_feats, B_feats, mutual_filter=True)
    A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
    B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

    num_corrs = A_corr.shape[1]
    print(f'NN generates {num_corrs} putative correspondences.')
    return A_corr, B_corr

In [ ]:
# visualize the point clouds together with feature correspondences
points = np.concatenate((A_corr.T,B_corr.T),axis=0)
lines = []
for i in range(num_corrs):
    lines.append([i,i+num_corrs])
colors = [[0, 1, 0] for i in range(len(lines))] # lines are shown in green
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([p0,p1,line_set])

In [ ]:
o3d.visualization.draw_geometries(boxes)

In [ ]:
r,h01 = register_two_views_teaser(boxes[0],boxes[1],0.05)
r,h12 = register_two_views_teaser(boxes[1],boxes[2],0.05)
r,h32 = register_two_views_teaser(boxes[3],boxes[2],0.05)
r,h43 = register_two_views_teaser(boxes[4],boxes[3],0.05)
r,h52 = register_two_views_teaser(boxes[5],boxes[2],0.05)

In [ ]:
t01 = colored_ICP(boxes[0],boxes[1])
draw_registration_result(boxes[0],boxes[1],t01)
t12 = colored_ICP(boxes[1],boxes[2])
draw_registration_result(boxes[1],boxes[2],t12)
t32 = colored_ICP(boxes[3],boxes[2])
draw_registration_result(boxes[3],boxes[2],t32)
t43 = colored_ICP(boxes[4],boxes[3])
draw_registration_result(boxes[4],boxes[3],t43)
t52 = colored_ICP(boxes[5],boxes[2])
draw_registration_result(boxes[5],boxes[2],t52)

In [ ]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])



In [ ]:
print("Statistical oulier removal")
cl, ind = pbc.remove_statistical_outlier(nb_neighbors=2,
                                                    std_ratio=0.5)
display_inlier_outlier(pbc, ind)



In [ ]:
pbc

In [ ]:
o3d.visualization.draw_geometries([cl])

In [ ]:
o3d.visualization.draw_geometries([pbc])

In [ ]:
for i in range(len(boxes)):
    o3d.visualization.draw_geometries([boxes[i]])

In [ ]:
o3d.visualization.draw_geometries([p0,p1,boxes[2],p3,p4,p5])

In [ ]:
point_clouds = [p0,p1,boxes[2],p3,p4,p5]

In [ ]:
kdtrees = [o3d.geometry.KDTreeFlann(pc) for pc in point_clouds]

In [ ]:
find_knn_cpu

In [ ]:
z

In [ ]:
x,y,z = kdtrees[0].search_knn_vector_3d([0,0,0],10)

In [ ]:
H0 = t12 @ t01
H1 = t12
H3 = t32
H4 = t32 @ t43
H5 = t52

p0 = copy.deepcopy(boxes[0]).transform(H0)
p1 = copy.deepcopy(boxes[1]).transform(H1)
p3 = copy.deepcopy(boxes[3]).transform(H3)
p4 = copy.deepcopy(boxes[4]).transform(H4)
p5 = copy.deepcopy(boxes[5]).transform(H5)

o3d.visualization.draw_geometries([p0,p1,boxes[2],p3,p4,p5])

In [ ]:
box_c1_combined = o3d.geometry.PointCloud()
box_c1_combined = p0+p1+boxes[2]+p3+p4+p5

In [ ]:
def compute_and_orient_normals(pcd, voxel_size):
    normal_radius = voxel_size*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius, max_nn=30))
    pcd.orient_normals_towards_camera_location()
    return pcd

In [ ]:
 o3d.io.write_point_cloud(box_path+"/c1.ply", pbc)

In [ ]:
pcd_cleaned_down = pcd_cleaned.voxel_down_sample(0.05)
o3d.visualization.draw_geometries([pcd_cleaned_down])

In [ ]:
print('run Poisson surface reconstruction')

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd_cleaned_down, depth=8)
o3d.visualization.draw_geometries([mesh])

In [ ]:
pcd_cleaned = o3d.io.read_point_cloud("/home/vigir3d/Desktop/c1_cleaned.ply")

In [ ]:
o3d.visualization.draw_geometries([pcd_cleaned])

In [ ]:
pbc = compute_and_orient_normals(box_c1_combined,0.01)

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])



In [ ]:
draw_registration_result_original_color(src,tgt,t)

In [ ]:
r,t = register_two_views_ateaser(src,tgt,0.01)

In [ ]:
src = copy.deepcopy(pcds[0])
tgt = copy.deepcopy(pcds[1])


radius = 0.01
print("3-2. Estimate normal.")
src.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

tgt.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))



sigma = 0.1
loss = o3d.pipelines.registration.GMLoss(k=sigma)
print("Using robust loss:", loss)
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane(loss)


threshold = 0.1
reg_p2p = o3d.pipelines.registration.registration_icp(
     src, tgt, threshold, np.eye(4), p2l,
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(pcds[0], pcds[1], reg_p2p.transformation)

In [ ]:
# register pairs and add:
r,t = register_two_views_teaser(pcd)

In [ ]:
print("Created " ,len(pcds), " point clouds")

t01= colored_ICP(pcds[0],pcds[1])
t52= colored_ICP(pcds[5],pcds[2])
t43= colored_ICP(pcds[4],pcds[3])
t12= colored_ICP(pcds[1],pcds[2])
t32= colored_ICP(pcds[3],pcds[2])


H0 = t12 @ t01
H1 = t12
H3 = t32
H4 = t32 @ t43
H5 = t52


np.savetxt("c_icp_0_2.txt", H0)
np.savetxt("c_icp_1_2.txt", H1)
np.savetxt("c_icp_3_2.txt", H3)
np.savetxt("c_icp_4_2.txt", H4)
np.savetxt("c_icp_5_2.txt", H5)

p0 = copy.deepcopy(pcds[0]).transform(H0)
p1 = copy.deepcopy(pcds[1]).transform(H1)
p3 = copy.deepcopy(pcds[3]).transform(H3)
p4 = copy.deepcopy(pcds[4]).transform(H4)
p5 = copy.deepcopy(pcds[5]).transform(H5)

o3d.visualization.draw_geometries([p0,p1,pcds[2],p3,p4,p5])

In [ ]:
def bilateral_filter(depth_image):
    # Assuming depth_image is your depth image as a numpy array
    # Make sure to convert it to an appropriate format (e.g., uint8) if it's not already
    depth_image = (depth_image / np.max(depth_image) * 255).astype(np.uint8)

    # Apply the bilateral filter
    filtered_image = cv2.bilateralFilter(depth_image, d=9, sigmaColor=75, sigmaSpace=75)

    # If you need the filtered depth map in its original range, you can scale it back:
    filtered_image = (filtered_image / 255.0 * np.max(depth_image)).astype(depth_image.dtype)
    return filtered_image

In [ ]:
bpath = '/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_cyl10/'
bpcds = generate_pcds(bpath,tforms,True)
for i in range(len(bpcds)):
   
    o3d.visualization.draw_geometries([bpcds[i]])

In [ ]:
o3d.visualization.draw_geometries(bpcds)

In [ ]:
r,t = register_two_views_teaser(bpcds[0],bpcds[1],0.05)
r,t = register_two_views_teaser(bpcds[1],bpcds[2],0.05)
r,t = register_two_views_teaser(bpcds[5],bpcds[2],0.05)
r,t = register_two_views_teaser(bpcds[4],bpcds[3],0.05)
r,t = register_two_views_teaser(bpcds[3],bpcds[5],0.05)


In [ ]:

t01= colored_ICP(bpcds[0],bpcds[1])
t52= colored_ICP(bpcds[5],bpcds[2])
t43= colored_ICP(bpcds[4],bpcds[3])
t12= colored_ICP(bpcds[1],bpcds[2])
t32= colored_ICP(bpcds[3],bpcds[2])


H0 = t12 @ t01
H1 = t12
H3 = t32
H4 = t32 @ t43
H5 = t52


p0 = copy.deepcopy(bpcds[0]).transform(H0)
p1 = copy.deepcopy(bpcds[1]).transform(H1)
p3 = copy.deepcopy(bpcds[3]).transform(H3)
p4 = copy.deepcopy(bpcds[4]).transform(H4)
p5 = copy.deepcopy(bpcds[5]).transform(H5)

o3d.visualization.draw_geometries([p0,p1,bpcds[2],p3,p4,p5])

In [ ]:
def colored_ICP(source, target):
    #draw_registration_result(source, target, np.eye(4))
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        print("iteration: ", iters, radius, scale)

        print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = copy.deepcopy(source).voxel_down_sample(radius)
        target_down = copy.deepcopy(target).voxel_down_sample(radius)

        print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = result_icp.transformation
    
    return current_transformation

In [6]:

# Apply the ICP results to new data


def generate_pcds(data_path, tforms, apply_tforms=True):  # returns a list of  pcds
    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    
    files = glob.glob(data_path+'/*.mkv')
    files.sort()
    print("There are : ", len(files), "present here!")

    list_size = len(files)
    pcds = [None] * list_size

    for i in range(len(files)): # for each view
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        print("Current File: ", file_name)

        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)
        K = (fx,fy,cx,cy)

        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame()
            if rgbda is None:
                print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            print("Got the last frame")
        else:
            print("************No valid frames found in the .mkv file.**********")
            
        #depth_image_array = np.asarray(last_frame.depth)
        #filtered_depth = eliminate_flying_pixels(depth_image_array, ws, flying_pixel_filter_threshold)

        #masked_rgbd = segment_images(last_frame)
        
        #pcd = backproject_o3d(masked_rgbd, camera_intrinsics)
        filt_pcd = backproject_o3d(last_frame, camera_intrinsics)
        #o3d.visualization.draw_geometries([pcd])
        #o3d.visualization.draw_geometries([filt_pcd])
        if apply_tforms:
            pcds[i] = copy.deepcopy(filt_pcd).transform(tforms[i])
        else :
            pcds[i] = filt_pcd
    return pcds

In [3]:
def read_sort_files(path):
    # Get a list of all files that start with 'H_' and end with '.txt'
    files = glob.glob(os.path.join(path, 'H_*.txt'))
    
    # Sort the list of files
    files.sort()
    
    # Load the sorted files as NumPy arrays and store them in a list
    arrays = [np.loadtxt(file) for file in files]
    
    return arrays

In [ ]:
H[0]

In [7]:
data_path_boxes = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_box_1'
calib_path = '/home/vigir3d/Software/programs/Cattle_Scanner'
#H = read_sort_files(calib_path)
#transforms = [H0,H1,np.eye(4), H3, H4, H5]
boxes = generate_pcds(data_path_boxes,tforms_apriltag,True)

NameError: name 'tforms_apriltag' is not defined

In [ ]:
boxes_combined = o3d.geometry.PointCloud()
for box in boxes :
    boxes_combined+=box

In [ ]:
pwd

In [ ]:
o3d.io.write_point_cloud("boxes_1.ply",boxes_combined)

In [ ]:
E0 = h12 @ h01
E1 = h12
E3 = h32
E4 = h32 @ h43
E5 = h52
pcds_combined = o3d.geometry.PointCloud()

b0 = copy.deepcopy(boxes[0]).transform(E0)
pcds_combined = b0
o3d.visualization.draw_geometries([pcds_combined])
b1 = copy.deepcopy(boxes[1]).transform(E1)
pcds_combined +=b1
o3d.visualization.draw_geometries([pcds_combined])
pcds_combined +=boxes[2]
o3d.visualization.draw_geometries([pcds_combined])
b3 = copy.deepcopy(boxes[3]).transform(E3)
pcds_combined +=b3
o3d.visualization.draw_geometries([pcds_combined])
b4 = copy.deepcopy(boxes[4]).transform(E4)
pcds_combined +=b4
o3d.visualization.draw_geometries([pcds_combined])
b5 = copy.deepcopy(boxes[5]).transform(E5)
pcds_combined +=b5
o3d.visualization.draw_geometries([pcds_combined])


In [ ]:
r, h01 = register_two_views_teaser(boxes[0],boxes[1],0.05)
r, h12 = register_two_views_teaser(boxes[1],boxes[2],0.05)
r, h32 = register_two_views_teaser(boxes[3],boxes[2],0.05)
r, h43 = register_two_views_teaser(boxes[4],boxes[3],0.05)
r, h52 = register_two_views_teaser(boxes[5],boxes[2],0.05)


In [ ]:
b01= colored_ICP(boxes[0],boxes[1])
b52= colored_ICP(boxes[5],boxes[2])
b43= colored_ICP(boxes[4],boxes[3])
b12= colored_ICP(boxes[1],boxes[2])
b32= colored_ICP(boxes[3],boxes[2])


T0 = b12 @ b01
T1 = b12
T3 = b32
T4 = b32 @ b43
T5 = b52

pcds_combined = o3d.geometry.PointCloud()

b0 = copy.deepcopy(boxes[0]).transform(T0)
pcds_combined = b0
o3d.visualization.draw_geometries([pcds_combined])
b1 = copy.deepcopy(boxes[1]).transform(T1)
pcds_combined +=b1
o3d.visualization.draw_geometries([pcds_combined])
pcds_combined +=boxes[2]
o3d.visualization.draw_geometries([pcds_combined])
b3 = copy.deepcopy(boxes[3]).transform(T3)
pcds_combined +=b3
o3d.visualization.draw_geometries([pcds_combined])
b4 = copy.deepcopy(boxes[4]).transform(T4)
pcds_combined +=b4
o3d.visualization.draw_geometries([pcds_combined])
b5 = copy.deepcopy(boxes[5]).transform(T5)
pcds_combined +=b5
o3d.visualization.draw_geometries([pcds_combined])


In [ ]:
o3d.visualization.draw_geometries([pcd01])
o3d.visualization.draw_geometries([pcd43])
o3d.visualization.draw_geometries([pcd52])

In [ ]:
pcd01 = combine_pairs(boxes[0], boxes[1], b01)
pcd43 = combine_pairs(boxes[4], boxes[3], b43)
pcd52 = combine_pairs(boxes[5], boxes[2], b52)


In [ ]:
pcd0152 = combine_pairs(pcd01,pcd52,t12)

In [ ]:
o3d.visualization.draw_geometries([p])

In [ ]:
tt = colored_ICP(pcd43,boxes[5])
draw_registration_result(pcd43,boxes[5],tt)

In [ ]:
#r12,t12 = register_two_views_teaser(pcd01,pcd52,0.08)
#tt = colored_ICP(pcd43,pcd0152)
#draw_registration_result(pcd43,pcd0152,tt)
for vs in np.arange(0.01,0.1,0.01):
    print("***", vs)
    r32,t32 = register_two_views_teaser(pcd43,boxes[5],vs)

In [ ]:
def combine_pairs(src,target,T):
    pcd_combined_pair = o3d.geometry.PointCloud()
    pcd_combined_pair = copy.deepcopy(src).transform(T) + target
    return pcd_combined_pair